In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/clean_data.csv')

In [3]:
df

,Unnamed: 0,club_name,player_name,age_x,position_x,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,...,CLaction,CLgoals,CLassists,CLMotM,CLrating,ELaction,ELgoals,ELassists,ELMotM,ELrating
0,0,FC Barcelona,Neymar,25.0,Left Winger,Paris Saint-Germain,£199.80m,out,Summer,199.8,...,31.0,17.0,5.0,7.0,246.27,0.0,0.0,0.0,0.0,0.00
1,1,AS Monaco,Kylian Mbappé,19.0,Centre-Forward,Paris Saint-Germain,£130.50m,out,Summer,130.5,...,9.0,6.0,0.0,1.0,64.43,1.0,0.0,0.0,0.0,6.09
2,2,Borussia Dortmund,Ousmane Dembélé,20.0,Right Winger,FC Barcelona,£124.20m,out,Summer,124.2,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3,3,SL Benfica,João Félix,19.0,Second Striker,Atlético Madrid,£113.40m,out,Summer,113.4,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
4,4,Atlético Madrid,Antoine Griezmann,28.0,Second Striker,FC Barcelona,£108.00m,out,Summer,108.0,...,46.0,20.0,3.0,6.0,332.84,8.0,6.0,2.0,3.0,63.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3165,4831,FC Utrecht,Rico Strieder,27.0,Defensive Midfield,PEC Zwolle,?,out,Summer,NaN,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3166,4832,Barnsley FC,Mallik Wilks,21.0,Left Winger,Hull City,?,out,Summer,NaN,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3167,4833,Derby County,Mason Bennett,24.0,Left Winger,Millwall FC,?,out,Summer,NaN,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00
3168,4834,Aston Villa,James Bree,22.0,Right-Back,Luton Town,?,out,Summer,NaN,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00


In [4]:
# Cleaning a little bit more

In [5]:
df = df.drop('dribledPast', axis = 1)
df = df[df.country != 'Montserrat'] # No country data
df = df[df.country != 'Guadeloupe'] # No country data

In [6]:
df.balance = df.balance.str.replace('k', '000').str.replace('m', '000000').str.replace('b', '000000000').str.replace('£', '').astype(int)
df.wage = df.wage.str.replace('k', '000').str.replace('m', '000000').str.replace('£', '').astype(int)
df.wagebudget = df.wagebudget.str.replace('k', '000').str.replace('m', '000000').str.replace('£', '').astype(int)
df.value = df.value.str.replace('k', '000').str.replace('m', '000000').str.replace('£', '').astype(int)
df.cost = df.cost.str.replace('k', '000').str.replace('m', '000000').str.replace('£', '').astype(int)
df.injprone = df.injprone.str.replace('%', '').astype(int)

In [7]:
# Contract duration
df['contract'] = df.expires.str.split('-', expand=True)[2].astype(int) - df.year_x + 0.5 * (df.expires.str.split('-', expand=True)[1] == '12')
df = df[df['contract'] >= 0]

In [8]:
# Remove unknown fees
df = df[df.fee != '?']

In [9]:
# Positional features
df['GK'] = df.position_x.str.contains('Goalkeeper')
df['DEF'] = df.position_x.str.contains('Back') | df.position_x.str.contains('Defensive')
df['MID'] = df.position_x.str.contains('Mid')
df['STR'] = df.position_x.str.contains('Striker') | df.position_x.str.contains('Forward') | df.position_x.str.contains('Winger')
df['Wing'] = df.position_x.str.contains('Winger') | df.position_x.str.contains('Right') | df.position_x.str.contains('Left')
df['Central'] = df.position_x.str.contains('Centr') | df.position_x.str.contains('Striker')

In [10]:
uefa_rankings = pd.DataFrame(
    [
        ['Primera Division', 1],
        ['Premier League', 2],
        ['Serie A', 3],
        ['1 Bundesliga', 4],
        ['Ligue 1', 5],
        ['Championship', 5.5],
        ['Liga Nos', 6],
        ['Premier Liga', 7],
        ['Eredivisie', 8]
    ], columns = ['league_name', 'UEFAranking']
)

In [11]:
# If player has no country, then country = country where currently playing
df.country = df.country.fillna(df.clubcountry)

In [12]:
df = df.merge(uefa_rankings, on='league_name', how='left')

In [13]:
# Adding Country Data

In [14]:
gdp = pd.read_csv('country data/GDPpc.csv')[['Country Name', '2019']].rename(columns={'Country Name':'country', '2019':'GDP'})
pop = pd.read_csv('country data/pop.csv')[['Country Name', '2019']].rename(columns={'Country Name':'country', '2019':'CountryPopulation'})
forex = pd.read_csv('country data/forex.csv')

In [15]:
df = df.merge(gdp, on='country', how='left')
df = df.merge(pop, on='country', how='left')
df = df.merge(forex, on='year', how='left')
df = df.drop(['year'], axis=1)

In [16]:
# Target engineering
df['fee_log'] = np.log(df.fee_cleaned + 1)

In [17]:
# Renaming variables
df = df.rename(columns={'age_x':'age', 'rating_x':'ratingFM', 'after_loan':'afterloan', 'rating_y':'ratingWS', 'year_x':'year'})

In [18]:
identifiers = ['player_name', 'club_involved_name', 'season']
player_var = ['age', 'wage', 'value', 'cost', 'ratingFM', 'potential', 'injprone', 'afterloan', 'contract']
team_var = ['reputation', 'balance', 'wagebudget', 'trainingfacilities', 'youthfacilities', 'youthacademy', 'stadiumcap', 'avgattendance']
stats_var = ['gamesPlayed', 'subedOn', 'MotM', 'ratingWS']
offensive_var = ['goalsScored', 'assists', 'shots', 'passes', 'passesSuccess', 'aerialsWon', 'keyPasses', 'dribbles', 'offsides', 'dispossessed', 'badTouch', 'crosses', 'longBalls', 'throughBalls']
defensive_var = ['yellow', 'red', 'tackles', 'interceptions', 'fouls', 'offsidesProvoked', 'clearances', 'blockedPass', 'ownGoal']
previous_var = ['ATaction', 'ATgoals', 'ATassists', 'ATshots', 'ATpasses', 'ATpassesSuccess', 'ATtackles', 'ATinterceptions', 'ATMotM', 'ATrating', 'CLaction', 'CLgoals', 'CLassists', 'CLMotM', 'CLrating', 'ELaction', 'ELgoals', 'ELassists', 'ELMotM', 'ELrating']
pos_var = ['GK', 'DEF', 'MID', 'STR', 'Wing', 'Central']
external_var = ['year', 'GDP', 'CountryPopulation', 'UEFAranking', 'eurgbp']
features = player_var + team_var + stats_var + offensive_var + defensive_var + previous_var + pos_var + external_var
labels = ['fee_cleaned', 'fee_log']

In [19]:
len(features)

75

In [20]:
# Avoid div by 0
df.gamesPlayed = [max(1, i) for i in df.gamesPlayed]
df.ATaction = [max(1, i) for i in df.ATaction]
df.CLaction = [max(1, i) for i in df.CLaction]
df.ELaction = [max(1, i) for i in df.ELaction]

In [21]:
# Normalize features to per-game
for feature in offensive_var + defensive_var + ['MotM', 'ratingWS']:
    df[feature] /= df.gamesPlayed
for feature in ['ATgoals', 'ATassists', 'ATshots', 'ATpasses', 'ATpassesSuccess', 'ATtackles', 'ATinterceptions', 'ATMotM', 'ATrating']:
    df[feature] /= df.ATaction
for feature in ['CLgoals', 'CLassists', 'CLMotM', 'CLrating']:
    df[feature] /= df.CLaction
for feature in ['ELgoals', 'ELassists', 'ELMotM', 'ELrating']:
    df[feature] /= df.ELaction

In [22]:
# Feature normalization -> to 0 mean, 1 variance
from sklearn.preprocessing import StandardScaler

In [23]:
df[features]

,age,wage,value,cost,ratingFM,potential,injprone,afterloan,contract,reputation,...,DEF,MID,STR,Wing,Central,year,GDP,CountryPopulation,UEFAranking,eurgbp
0,25.0,495000,62000000,200000000,90.00,91.50,30,0,4.0,187.0,...,False,False,True,True,False,2017,11121.739915,211049527.0,1.0,0.8775
1,19.0,308000,53000000,151000000,78.60,-10.00,30,1,0.0,172.0,...,False,False,True,False,True,2018,44317.392315,67059887.0,5.0,0.8842
2,20.0,50000,25000000,121000000,75.30,-95.00,25,0,4.0,159.0,...,False,False,True,True,False,2017,44317.392315,67059887.0,4.0,0.8775
3,19.0,4000,9000000,52000000,79.80,92.30,20,0,4.0,163.0,...,False,False,True,False,True,2019,24590.425797,10269417.0,6.0,0.8964
4,28.0,791000,74000000,120000000,94.70,95.70,25,0,4.0,174.0,...,False,False,True,False,True,2019,44317.392315,67059887.0,1.0,0.8964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3002,31.0,180000,30000000,7000000,84.00,87.60,50,1,0.0,162.0,...,False,True,False,False,False,2020,4732.069506,2957731.0,2.0,0.9018
3003,25.0,40000,8000000,14000000,73.50,76.00,50,1,0.0,111.0,...,False,True,False,False,False,2016,14597.308096,3461734.0,2.0,0.8395
3004,31.0,31000,3000000,7000000,76.58,76.58,0,1,0.0,134.0,...,False,False,True,False,True,2014,35613.842286,60297396.0,3.0,0.7921
3005,30.0,15000,3000000,1000000,68.22,68.90,0,0,0.0,115.0,...,True,False,False,True,False,2014,43688.437455,66834405.0,5.5,0.7921


In [24]:
scaler = StandardScaler()
scaler.fit(df[features])
df[features] = scaler.transform(df[features])

In [25]:
df = df[identifiers + features + labels].dropna()

In [26]:
df

,player_name,club_involved_name,season,age,wage,value,cost,ratingFM,potential,injprone,...,STR,Wing,Central,year,GDP,CountryPopulation,UEFAranking,eurgbp,fee_cleaned,fee_log
0,Neymar,Paris Saint-Germain,2017/2018,-0.585046,12.315201,7.299882,11.404813,3.330984,0.957797,-0.151163,...,1.509692,1.526437,-0.974713,0.053782,-1.434860,3.183356,-1.703498,0.565087,199.8,5.302309
1,Kylian Mbappé,Paris Saint-Germain,2018/2019,-2.049648,7.434713,6.148324,8.490667,1.275699,-3.630068,-0.151163,...,1.509692,-0.655120,1.025943,0.565559,0.437768,0.177493,0.183647,0.668371,130.5,4.879007
2,Ousmane Dembélé,FC Barcelona,2017/2018,-1.805548,0.701205,2.565702,6.706495,0.680749,-7.472122,-0.377982,...,1.509692,1.526437,-0.974713,0.053782,0.437768,0.177493,-0.288139,0.565087,124.2,4.829912
3,João Félix,Atlético Madrid,2019/2020,-2.049648,-0.499342,0.518489,2.602902,1.492045,0.993957,-0.604802,...,1.509692,-0.655120,1.025943,1.077336,-0.675067,-1.008040,0.655433,0.856442,113.4,4.739701
4,Antoine Griezmann,FC Barcelona,2019/2020,0.147256,20.040466,8.835291,6.647023,4.178338,1.147640,-0.377982,...,1.509692,-0.655120,1.025943,1.077336,0.437768,0.177493,-1.703498,0.856442,108.0,4.691348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,Daniele Capelli,AC Cesena,2015/2016,0.391356,-0.368848,-0.505117,-0.430189,-0.451461,0.014912,1.436573,...,-0.662387,-0.655120,1.025943,-0.969772,-0.053215,0.036322,-0.759925,-1.964614,0.0,0.000000
3002,Henrikh Mkhitaryan,AS Roma,2020/2021,0.879557,4.094058,3.205456,-0.073354,2.249255,0.781514,0.756115,...,-0.662387,-0.655120,-0.974713,1.589113,-1.795314,-1.160675,-1.231711,0.939686,0.0,0.000000
3003,Gastón Ramírez,Middlesbrough FC,2016/2017,-0.585046,0.440217,0.390539,0.342952,0.356230,0.257187,0.756115,...,-0.662387,-0.655120,-0.974713,-0.457995,-1.238797,-1.150154,-1.231711,-0.020706,0.0,0.000000
3004,Antonio Cassano,Parma FC,2014/2015,0.879557,0.205327,-0.249215,-0.073354,0.911517,0.283403,-1.512079,...,1.509692,-0.655120,1.025943,-1.481549,-0.053215,0.036322,-0.759925,-0.751406,0.0,0.000000


In [27]:
df.to_csv('data/data.csv')